# 样例介绍

功能：对图像中的细胞核进行分割  
样例输入：未标注的生物细胞图像  
样例输出：已经分割的细胞核图像

# 前期准备

基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：
1. 获取模型和测试数据。我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，将模型和测试数据下载到本地，并放到本样例目录下。
2. 模型转换。利用atc工具将原始模型转换为om模型，转换命令如下：  
    ```shell
    atc --framework=5 --model=./unetplusplus.onnx --input_format=NCHW --input_shape="actual_input_1:1,3,96,96" --output=unetplusplus --log=error --soc_version=Ascend310B1
    ```

    其中各个参数具体含义如下：
    * --framework：原始框架类型,  0: Caffe, 1: MindSpore, 3: TensorFlow, 5: ONNX。
    * --model：原始onnx模型文件。
    * --input_format：输入Tensor的内存排列方式。
    * --input_shape：模型输入节点名称和shape。
    * --output：输出的模型文件路径。
    * --log：打印的日志级别。error表示只输出error类别的信息。
    * --soc_version：昇腾AI处理器型号，此处为"Ascend310B1"。

# 模型推理实现

### 1. 导入三方库

In [ ]:
import cv2
import numpy as np
from albumentations.augmentations import transforms
import matplotlib.pyplot as plt

import acl
import acllite_utils as utils
from acllite_model import AclLiteModel
from acllite_resource import resource_list

### 2. 定义acllite资源初始化与去初始化类

In [ ]:
class AclLiteResource:
    """
    AclLiteResource
    """
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """
        init resource
        """
        print("init resource stage:")
        ret = acl.init()

        ret = acl.rt.set_device(self.device_id)
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id)
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream()
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode()
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream)

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context)

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id)
        print("Release acl resource success")

### 3. 定义前后处理所需要的函数

In [ ]:
def sigmoid(x):
    y = 1.0 / (1 + np.exp(-x))  # 对矩阵的每个元素执行 1/(1+e^(-x))
    return y


def save_color_png(img, msk, color):
    """ 将推理得到的 mask 覆盖到原图上 """
    msk = msk + 0.5  # 将像素值范围变换到 0.5~1.5，有利于下面转为二值图
    msk = cv2.resize(msk, (img.shape[1], img.shape[0]))  # 将 mask 缩放到原图大小
    msk = np.array(msk, np.uint8)  # 转为二值图，只包含 0 和 1
    contours, _ = cv2.findContours(msk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if color=='red':
        cv2.drawContours(img, contours, -1, (0, 0, 255), 1) 
        img[..., 2] = np.where(msk == 1, 255, img[..., 2])  # 将轮廓线以内（即分割区域）覆盖上一层红色

    return img

### 4. 构造主函数，串联整个代码逻辑

In [ ]:
def main():
    pic_input = 'img.png'  # 单张图片
    model_path = "unetplusplus.om"  # 模型路径

    # 类别数量
    num_class = 1 

    # acl初始化
    acl_resource = AclLiteResource()
    acl_resource.init()

    # 加载模型
    model = AclLiteModel(model_path)  

    # 读入图片
    img_bgr = cv2.imread(pic_input)

    # 前处理
    img = cv2.resize(img_bgr, (96, 96))  # 将原图缩放到 96*96 大小
    nor = transforms.Normalize()  
    img = nor.apply(img)  # 将像素值归一化（减去均值除以方差）
    img = img.astype('float32') / 255  # 将像素值缩放到 0~1 范围内
    img = img.transpose(2, 0, 1)  # 将形状转换为 channel first (3, 96, 96)

    # 推理
    model_out = model.execute([img, ])  # 将图片放入模型进行推理，输出为列表形式，若模型只有一个输出，则只包含一个元素
    model_out_msk = model_out[0]  # 取出模型推理结果，推理结果形状为 (1, 1, 96, 96),即（batchsize, num_class, height, width）

    # 后处理
    color_list = ['red']  # 显示推理结果时覆盖的颜色
    model_out_msk = sigmoid(model_out_msk[0][0])  # 将模型输出变换到 0~1 范围内
    img_to_save = save_color_png(img_bgr, model_out_msk, color_list[0])  # 将处理后的输出画在原图上，并返回

    # 画出推理结果
    plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    img_to_save = img_to_save[:,:,[2,1,0]]
    plt.imshow(img_to_save)

### 5. 运行
运行完成后，会显示推理后的图片，覆盖为红色的部分即为分割出的细胞核。

In [ ]:
main()

# 样例总结

我们来回顾一下以上代码，可以包括以下几个步骤：
1. 初始化acl资源：在调用acl相关资源时，必须先初始化AscendCL，否则可能会导致后续系统内部资源初始化出错。  
2. 对图片进行前处理：在此样例中，我们首先利用opencv读入图片，再对图片进行缩放、标准化、维度转换等操作。
3. 推理：利用AclLiteModel.execute接口对图片进行推理。    
4. 对推理结果进行后处理：把模型输出mask利用sigmoid压缩到0-1范围内，再检测其轮廓，最后将分割区域（轮廓内部）填充为红色。  
5. 可视化图片：在界面画出填充颜色后的分割结果。